In [66]:
import os
from src.config import config
import polars as pl

def read_pbp_agg(run_id: str) -> pl.DataFrame:
    pbp_agg_path = config['local']['data_paths']['outputs']['play_by_play_agg']
    pbp_filename = f'play_by_play_agg_{run_id}.parquet'
    return pl.read_parquet(os.path.join(pbp_agg_path, pbp_filename))


In [70]:

df = read_pbp_agg('20240809')
df

game_id,team,home_away,opponent,player,player_id,passing_attempts,passing_touchdowns,passing_yards,rushing_attempts,rushing_yards,rushing_touchdowns,receptions,receiving_yards,receiving_touchdowns,fumbles,interceptions,season,week,position,curr_team
str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,i32,f64,f64,f64,f64,i64,i64,str,str
"""2020_07_JAX_LAC""","""LAC""","""home""","""LAC""","""M.Williams""","""00-0033536""",0.0,0.0,0.0,0.0,0.0,0.0,3,4.0,0.0,0.0,0.0,2020,7,"""WR""","""NYJ"""
"""2022_12_PIT_IND""","""PIT""","""away""","""PIT""","""A.McFarland""","""00-0036336""",0.0,0.0,0.0,6.0,30.0,0.0,2,11.0,0.0,0.0,0.0,2022,12,"""RB""","""PIT"""
"""2022_19_LAC_JAX""","""LAC""","""away""","""LAC""","""J.Palmer""","""00-0036988""",0.0,0.0,0.0,0.0,0.0,0.0,6,31.0,0.0,0.0,0.0,2022,19,"""WR""","""LAC"""
"""2021_19_PIT_KC""","""PIT""","""away""","""PIT""","""B.Roethlisberger""","""00-0022924""",46.0,2.0,201.0,2.0,-1.0,0.0,0,0.0,0.0,2.0,0.0,2021,19,"""QB""","""PIT"""
"""2023_05_JAX_BUF""","""BUF""","""home""","""BUF""","""D.Knox""","""00-0035689""",0.0,0.0,0.0,0.0,0.0,0.0,6,17.0,0.0,0.0,0.0,2023,5,"""TE""","""BUF"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2022_10_LAC_SF""","""LAC""","""away""","""LAC""","""A.Ekeler""","""00-0033699""",0.0,0.0,0.0,6.0,24.0,0.0,12,39.0,0.0,0.0,0.0,2022,10,"""RB""","""WAS"""
"""2021_13_ARI_CHI""","""ARI""","""away""","""ARI""","""D.Hopkins""","""00-0030564""",0.0,0.0,0.0,0.0,0.0,0.0,2,32.0,1.0,0.0,0.0,2021,13,"""WR""","""TEN"""
"""2022_06_CIN_NO""","""CIN""","""away""","""CIN""","""H.Hurst""","""00-0034830""",0.0,0.0,0.0,0.0,0.0,0.0,3,21.0,0.0,0.0,0.0,2022,6,"""TE""","""LAC"""


In [42]:
from src.utils.points_calc import calculate_fantasy_points
from src.utils.points_config import STANDARD_HALF_PPR

    # df['fantasy_points'] = df.apply(lambda x: calculate_fantasy_points(STANDARD_HALF_PPR, x.passing_yards, x.passing_touchdowns,
    #                                                                    x.interceptions, x.receptions, x.receiving_yards,
    #                                                                    x.receiving_touchdowns, x.rushing_yards,
    #                                                                    x.rushing_touchdowns, x.fumbles), axis=1)

df.with_columns(pl.struct(['game_id', 'passing_touchdowns', 'interceptions', 'receptions', 'receiving_yards', 'receiving_touchdowns', 'rushing_yards', 'rushing_touchdowns', 'fumbles']).apply(lambda row: calculate_fantasy_points(STANDARD_HALF_PPR, x['passing_yards'])))

AttributeError: 'Expr' object has no attribute 'apply'

In [71]:
df.with_columns(pl.struct('passing_yards', 'passing_touchdowns', 'interceptions', 'receptions', 'receiving_yards', 'receiving_touchdowns', 'rushing_yards', 'rushing_touchdowns', 'fumbles') \
                .map_elements(lambda x: calculate_fantasy_points(STANDARD_HALF_PPR, x['passing_yards'], x['passing_touchdowns'],
x['interceptions'], x['receptions'], x['receiving_yards'], x['receiving_touchdowns'], x['rushing_yards'], x['rushing_touchdowns'], x['fumbles']), return_dtype=pl.Float64).alias('fantasy_points'))

game_id,team,home_away,opponent,player,player_id,passing_attempts,passing_touchdowns,passing_yards,rushing_attempts,rushing_yards,rushing_touchdowns,receptions,receiving_yards,receiving_touchdowns,fumbles,interceptions,season,week,position,curr_team,fantasy_points
str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,i32,f64,f64,f64,f64,i64,i64,str,str,f64
"""2020_07_JAX_LAC""","""LAC""","""home""","""LAC""","""M.Williams""","""00-0033536""",0.0,0.0,0.0,0.0,0.0,0.0,3,4.0,0.0,0.0,0.0,2020,7,"""WR""","""NYJ""",1.9
"""2022_12_PIT_IND""","""PIT""","""away""","""PIT""","""A.McFarland""","""00-0036336""",0.0,0.0,0.0,6.0,30.0,0.0,2,11.0,0.0,0.0,0.0,2022,12,"""RB""","""PIT""",5.1
"""2022_19_LAC_JAX""","""LAC""","""away""","""LAC""","""J.Palmer""","""00-0036988""",0.0,0.0,0.0,0.0,0.0,0.0,6,31.0,0.0,0.0,0.0,2022,19,"""WR""","""LAC""",6.1
"""2021_19_PIT_KC""","""PIT""","""away""","""PIT""","""B.Roethlisberger""","""00-0022924""",46.0,2.0,201.0,2.0,-1.0,0.0,0,0.0,0.0,2.0,0.0,2021,19,"""QB""","""PIT""",5.91
"""2023_05_JAX_BUF""","""BUF""","""home""","""BUF""","""D.Knox""","""00-0035689""",0.0,0.0,0.0,0.0,0.0,0.0,6,17.0,0.0,0.0,0.0,2023,5,"""TE""","""BUF""",4.7
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2022_10_LAC_SF""","""LAC""","""away""","""LAC""","""A.Ekeler""","""00-0033699""",0.0,0.0,0.0,6.0,24.0,0.0,12,39.0,0.0,0.0,0.0,2022,10,"""RB""","""WAS""",12.3
"""2021_13_ARI_CHI""","""ARI""","""away""","""ARI""","""D.Hopkins""","""00-0030564""",0.0,0.0,0.0,0.0,0.0,0.0,2,32.0,1.0,0.0,0.0,2021,13,"""WR""","""TEN""",10.2
"""2022_06_CIN_NO""","""CIN""","""away""","""CIN""","""H.Hurst""","""00-0034830""",0.0,0.0,0.0,0.0,0.0,0.0,3,21.0,0.0,0.0,0.0,2022,6,"""TE""","""LAC""",3.6


In [62]:
df.select(pl.struct(['passing_yards', 'passing_touchdowns', 'interceptions', 'receptions', 'receiving_yards', 'receiving_touchdowns', 'rushing_yards', 'rushing_touchdowns', 'fumbles']))

ColumnNotFoundError: passing_yards

In [59]:
df = pl.DataFrame({"keys": ["a", "a", "b"], "values": [10, 7, 1]})

df

keys,values
str,i64
"""a""",10
"""a""",7
"""b""",1


In [61]:
df.select(
    pl.struct(["keys", "values"]))

keys
struct[2]
"{""a"",10}"
"{""a"",7}"
"{""b"",1}"


In [60]:
out = df.select(
    pl.struct(["keys", "values"])
    .map_elements(lambda x: len(x["keys"]) + x["values"], return_dtype=pl.Int64)
    .alias("solution_map_elements"),
    (pl.col("keys").str.len_bytes() + pl.col("values")).alias("solution_expr"),
)
print(out)

shape: (3, 2)
┌───────────────────────┬───────────────┐
│ solution_map_elements ┆ solution_expr │
│ ---                   ┆ ---           │
│ i64                   ┆ i64           │
╞═══════════════════════╪═══════════════╡
│ 11                    ┆ 11            │
│ 8                     ┆ 8             │
│ 2                     ┆ 2             │
└───────────────────────┴───────────────┘
